In [ ]:
from fastai.vision.all import *
from fastai.vision.widgets import *

In [ ]:
from bing_image_downloader import downloader
for q in ['breakfast','lunch', 'dinner']:
    downloader.download(f'{q} food', limit=150, output_dir='meals', adult_filter_off=True, force_replace=False, timeout=5)

In [ ]:
types = 'breakfast','lunch', 'dinner'
path = Path('meals')

In [ ]:
fns = get_image_files(path)
fns

In [ ]:
failed = verify_images(fns)
failed

In [ ]:
failed.map(Path.unlink)

In [ ]:
meals = DataBlock(
    blocks=(ImageBlock, CategoryBlock),  
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
dls = meals.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=4, nrows=1)

In [ ]:
meals = meals.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = meals.dataloaders(path)

In [ ]:
dls.train.show_batch(max_n=8, nrows=2, unique=True)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(1, nrows=1)

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.export()